In [ ]:
# include("../finite_difference.jl")
# include("../ode_solver.jl")

# kappa = 1   # diffusion constant
# L = 1.0         # length of spatial domain
# T = 0.5         # total time to solve for

# function u_I(x)
#     # initial temperature distribution
#     y = sin.(pi*x/L)
#     return y
# end

# function u_exact(x,t)
#     # the exact solution
#     y = exp.(-kappa*(pi^2/L^2)*t)*sin.(pi*x/L)
#     return y
# end

# # Set numerical parameters
# mx = 10     # number of gridpoints in space
# mt = 1000   # number of gridpoints in time


# # forward euler Estimate
# weird_x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, forward_euler)
# # Create trace
# f_euler = scatter(x=x, y=u_j, mode="markers", name="forward euler", showlegend=true)


# # backward euler Estimate
# x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, backward_euler)
# # Create trace
# b_euler = scatter(x=x, y=u_j, mode="markers", name="backward euler", showlegend=true)


# # crank nicholson Estimate
# x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, crank_nicholson)
# # Create trace
# c_nicholson = scatter(x=x, y=u_j, mode="markers", name="crank nicholson", showlegend=true)



# # Plot the final result and exact solution
# xx = LinRange(0,L,250)

# # Create solution trace
# exact = scatter(x=xx, y=u_exact(xx,T), mode="lines", name="exact", showlegend=true)

# layout = Layout(
#     xaxis_title = "x",
#     yaxis_title = "u(x,0.5)"
#     )

# plot([exact, f_euler, b_euler, c_nicholson], layout)

In [149]:
using PlotlyJS
using Test
include("../ode_solver.jl")
include("../numerical_continuation.jl")
include("../numerical_shooting.jl")
include("../examples/example_functions.jl")
include("../visualisation.jl")
include("../finite_difference.jl")

crank_nicholson (generic function with 1 method)

In [150]:
continuation(hopf2d, [1 1], 6, "beta", 0:0.01:2, method="pseudo_arclength",discretisation="shooting")

([0.0, 0.01, 0.020130110920027276, 0.03026022184005455, 0.040390332760081825, 0.0505204436801091, 0.060650554600136374, 0.07078066552016365, 0.08091077644019093, 0.0910408873602182  …  1.9144608529616745, 1.9245909638816818, 1.9347210748016892, 1.9448511857216966, 1.954981296641704, 1.9651114075617113, 1.9752415184817187, 1.985371629401726, 1.9955017403217334, 2.005631851241741], [-0.0011406615456035674 -9.346260637541235e-14 6.283184897710069; -1.3236927973877773e-15 -1.196504658541496e-18 6.2831849133496345; … ; -2.583180594980991e-15 -3.5480562425749724e-16 6.283394919387156; -2.5974921513389446e-15 -3.567711162738994e-16 6.283395990862821])

In [47]:
@testset verbose = true "System Unit Tests" begin

    @testset verbose = true "ode_solver" begin

        @testset verbose = true "Input Tests" begin

            # test error is thrown if t=0 is not included in t
            t = 0:0.1:1
            bad_t = 1:0.1:2
            @test_throws ErrorException solve_ode(f2, [1], bad_t, method="rk4")


            # test error is thrown if x0 is not a matrix
            @test_throws ErrorException solve_ode(f2, 1, t, method="rk4")
            @test_throws ErrorException solve_ode(f2, [1;0], t, method="rk4")
            @test_throws ErrorException solve_ode(f2, [1, 0], t, method="rk4")

        
            # test error is thrown if x0 is not a 1xn matrix
            @test_throws ErrorException solve_ode(f2, [1 2; 3 4], t, method="rk4")


            # test error is thrown if x0 is not correct length
            @test_throws ErrorException solve_ode(f2, [1], t, method="rk4")

        end

        @testset verbose = true "Output Tests" begin

            # test if solve_ode estimates a simple ODE correctly
            x0 = [1]
            t = 0:1
            e_estimate = solve_ode(f, x0, t, method="rk4")[end][1]
            @test isapprox(e_estimate, ℯ)


            # test if solve_ode estimates systems of ODEs correctly
            x0 = [1 0]
            t = 0:0.1:1
            f2_sol = f2_solution(x0, t)
            f2_numerical_sol = solve_ode(f2, x0, t, method="rk4")
            @test  all(isapprox.(f2_numerical_sol, f2_sol, atol=1e-6))

        end
    end

    @testset verbose = true "numerical_shooting" begin

        @testset verbose = true "Input Tests" begin

            # test error is thrown if T is not an integer or float
            T = 10
            bad_T = 1:0.1:2
            @test_throws ErrorException find_limit_cycle(f2, [1], bad_T)
        
        
            # test error is thrown if u0 is not a matrix
            @test_throws ErrorException find_limit_cycle(f2, 1, T)
            @test_throws ErrorException find_limit_cycle(f2, [1;0], T)
            @test_throws ErrorException find_limit_cycle(f2, [1, 0], T)
        
        
            # test error is thrown if u0 is not a 1xn matrix
            @test_throws ErrorException find_limit_cycle(f2, [1 2; 3 4], T)
        
        
            # test error is thrown if u0 is not correct length
            @test_throws ErrorException find_limit_cycle(f2, [1], 10)
        
        
            # test error is thrown if phase_index is a positive integer
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=-1)
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=[3])
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=2.5)
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index="0")
        
        end

        @testset verbose = true "Output Tests" begin

            # test if found limit cycle matches the analytical solution
            u0, T = find_limit_cycle(hopf2d, [-1 0], 6)
            @test isapprox(T, 2*pi)


            # test if solve_ode estimates a Hopf ODE correctly
            t = 0:0.1:T
            hopf_solution = hopf2d_sol(t, beta=1, theta=pi) # adjusted for phase
            hopf_numerical_sol = solve_ode(hopf2d, u0, t, method="rk4")
            @test  all(isapprox.(hopf_numerical_sol, hopf_solution, atol=1e-6))


            # test if arguments are being passed to the function correctly
            u0_0, T = find_limit_cycle(hopf2d, [-1 0], 6)
            u0_1, T = find_limit_cycle(hopf2d, [-1 0], 6, beta = 2)
            u0_3, T = find_limit_cycle(hopf2d, [-1 0], 6, beta = 2, sigma=-1.2)
            u0_2, T = find_limit_cycle(hopf2d, [-1 0], 6, sigma=-1.2)
            @test u0_0 != u0_1
            @test u0_0 != u0_2
            @test u0_0 != u0_3

        end

        @testset verbose = true "Higher Dimensional Output" begin

            # test if found limit cycle matches the analytical solution
            u0, T = find_limit_cycle(hopf3d, [1 1 1], 6)
            @test isapprox(T, 2*pi)


            u0, T = find_limit_cycle(hopf3d, [1 1 1], 10)
            hopf_solution = hopf3d_sol(u0, 0:0.1:T; theta=0) # adjusted for phase
            hopf_numerical_sol = solve_ode(hopf3d, u0, 0:0.1:T, method="rk4")
            @test all(isapprox.(hopf_numerical_sol, hopf_solution, atol=1e-5))
            

        end
    end

    @testset verbose = true "numerical_continuation" begin

        @testset verbose = true "Input Tests" begin

            # test error is thrown if T is not an integer or float
            T = 10
            bad_T = 1:0.1:2
            @test_throws ErrorException find_limit_cycle(f2, [1], bad_T)
        
        
            # test error is thrown if u0 is not a matrix
            @test_throws ErrorException find_limit_cycle(f2, 1, T)
            @test_throws ErrorException find_limit_cycle(f2, [1;0], T)
            @test_throws ErrorException find_limit_cycle(f2, [1, 0], T)
        
        
            # test error is thrown if u0 is not a 1xn matrix
            @test_throws ErrorException find_limit_cycle(f2, [1 2; 3 4], T)
        
        
            # test error is thrown if u0 is not correct length
            @test_throws ErrorException find_limit_cycle(f2, [1], 10)
        
        
            # test error is thrown if phase_index is a positive integer
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=-1)
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=[3])
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index=2.5)
            @test_throws ErrorException find_limit_cycle(f2, [1 1], 10, phase_index="0")

        end

        @testset verbose = true "Output Tests" begin
            
            # test if found limit cycle matches the analytical solution
            u0, T = find_limit_cycle(hopf2d, [-1 0], 6)
            @test isapprox(T, 2*pi)

        end
    end

    # @testset verbose = true "finite_difference" begin

    #     @testset verbose = true "Input Tests" begin

    #         # test error is thrown if t=0 is not included in t
    #         t = 0:0.1:1
    #         bad_t = 1:0.1:2
    #         @test_throws ErrorException solve_ode(f2, [1], bad_t, method="rk4")

    #     end

    #     @testset verbose = true "Output Tests" begin
            
    #         # test if found limit cycle matches the analytical solution
    #         u0, T = find_limit_cycle(hopf2d, [-1 0], 6)
    #         @test isapprox(T, 2*pi)

    #     end
    # end
end



Test Summary:                 | Pass  Total
Systems Tests                 |   27     27
  ode_solver                  |    8      8
    Input Tests               |    6      6
    Output Tests              |    2      2
  numerical_shooting          |   17     17
    Input Tests               |   10     10
    Output Tests              |    5      5
    Higher Dimensional Output |    2      2
  finite_difference           |    2      2
    Input Tests               |    1      1
    Output Tests              |    1      1


Test.DefaultTestSet("Systems Tests", Any[Test.DefaultTestSet("ode_solver", Any[Test.DefaultTestSet("Input Tests", Any[], 6, false, true), Test.DefaultTestSet("Output Tests", Any[], 2, false, true)], 0, false, true), Test.DefaultTestSet("numerical_shooting", Any[Test.DefaultTestSet("Input Tests", Any[], 10, false, true), Test.DefaultTestSet("Output Tests", Any[], 5, false, true), Test.DefaultTestSet("Higher Dimensional Output", Any[], 2, false, true)], 0, false, true), Test.DefaultTestSet("finite_difference", Any[Test.DefaultTestSet("Input Tests", Any[], 1, false, true), Test.DefaultTestSet("Output Tests", Any[], 1, false, true)], 0, false, true)], 0, false, true)

In [143]:

@testset verbose = true "numerical_continuation" begin

    @testset verbose = true "Input Tests" begin

        # test error is thrown if T is not an integer or float
        T = 10
        bad_T = 1:0.1:2
        @test_throws ErrorException pseudo_arclength(f2, [1], bad_T)
    
    
        # test error is thrown if u0 is not a matrix
        @test_throws ErrorException pseudo_arclength(f2, 1, T)
        @test_throws ErrorException pseudo_arclength(f2, [1;0], T)
        @test_throws ErrorException pseudo_arclength(f2, [1, 0], T)
    
    
        # test error is thrown if u0 is not a 1xn matrix
        @test_throws ErrorException pseudo_arclength(f2, [1 2; 3 4], T)
    
    
        # test error is thrown if u0 is not correct length
        @test_throws ErrorException pseudo_arclength(f2, [1], 10)
    
    
        # test error is thrown if phase_index is a positive integer
        @test_throws ErrorException pseudo_arclength(f2, [1 1], 10, phase_index=-1)
        @test_throws ErrorException pseudo_arclength(f2, [1 1], 10, phase_index=[3])
        @test_throws ErrorException pseudo_arclength(f2, [1 1], 10, phase_index=2.5)
        @test_throws ErrorException pseudo_arclength(f2, [1 1], 10, phase_index="0")

    end

    @testset verbose = true "Output Tests" begin
        
        # test if found limit cycle matches the analytical solution
        u0, T = find_limit_cycle(hopf2d, [-1 0], 6)
        @test isapprox(T, 2*pi)

    end
end

Test Summary:          | Pass  Total
numerical_continuation |   11     11
  Input Tests          |   10     10
  Output Tests         |    1      1


Test.DefaultTestSet("numerical_continuation", Any[Test.DefaultTestSet("Input Tests", Any[], 10, false, true), Test.DefaultTestSet("Output Tests", Any[], 1, false, true)], 0, false, true)

In [142]:
include("../numerical_continuation.jl")


pseudo_arclength (generic function with 1 method)